In [ ]:
# SPL - Private to Private – Domestic Trusted Group

!pip install phe
!pip install pycryptodome
!pip install faker

from phe import paillier
import pandas as pd
import random
from faker import Faker

# Generate synthetic SPL data
def generate_synthetic_spl(num_entries):
    fake = Faker()
    synthetic_spl = []
    for _ in range(num_entries):
        person = {
            "customer_id": ''.join(random.choices('ABCDEFGHJKLMNPQRSTUVWXYZ0123456789', k=10)),
            "name": fake.name(),
            "dob": fake.date_of_birth(minimum_age=18, maximum_age=90).strftime("%Y-%m-%d"),
            "ssn": fake.ssn(),
            "address": fake.address(),
            "transaction_activity": fake.random_int(min=1000, max=100000),
            "risk_score": random.randint(0, 100)
        }
        synthetic_spl.append(person)
    return pd.DataFrame(synthetic_spl)

# Generate SPL data
spl_institution1 = generate_synthetic_spl(5000)
spl_institution2 = generate_synthetic_spl(5000)

# Generate Paillier keys
public_key, private_key = paillier.generate_paillier_keypair()

# Encrypt SPL data
def encrypt_data(data, key):
    return [key.encrypt(value) for value in data]

# Encrypt risk scores
spl1_risk_scores = encrypt_data(spl_institution1['risk_score'], public_key)
spl2_risk_scores = encrypt_data(spl_institution2['risk_score'], public_key)

# Compare encrypted data
def compare_encrypted_data(encrypted_data1, encrypted_data2):
    matches = []
    for idx1, encrypted_value1 in enumerate(encrypted_data1):
        for idx2, encrypted_value2 in enumerate(encrypted_data2):
            if encrypted_value1 == encrypted_value2:
                matches.append((idx1, idx2))
    return matches

matches = compare_encrypted_data(spl1_risk_scores, spl2_risk_scores)
print("Count of matched items:", len(matches))
# print(f"Matches found: {matches}")
